# ASE Calculators

In this tutorial we will look at ASE's ability to run DFT calculations through python. This power is tremendous. It'll speed up our workflow by allowing us to control our entire DFT calculation using Python, without the need to write individual files for JDFTx.

Frome ASE's documentation:
"For ASE, a calculator is a black box that can take atomic numbers and atomic positions from an ```Atoms``` object and calculate the energy and forces and sometimes also stresses."

We will go through the general setup of an ASE calculator and then write a script to run a single point with JDFTx using the ASE calculator.

The general format for running a DFT calculation in ASE is to first read in an ```Atoms``` object and append a calculator to a structure. The pseudocode looks like this:
```python
from ASE import Calculator
from ASE import read

structure = read("POSCAR")
structure.calc = Calculator(DFT_options)
```

Then, we take an optimizer from ASE which is a prebuilt algorithm used to optimize the structure. We pass this optimizer our DFT calculation parameters, and run it.
```python
from ASE import Optimizer

options = calculation_parameters
opt = Optimizer
dyn = opt(atoms, options)
dyn.run(fmax, max_steps)
```

#### What's actually happening

The ```Calculator``` object that we import from ASE is an electronic structure calculator that uses some other DFT software to do the calculation. For example, one calculator can actually use VASP to perform electronic structure calculations. You can look at the [ASE VASP](https://wiki.fysik.dtu.dk/ase/ase/calculators/vasp.html#module-ase.calculators.vasp) documentation to see how to use it.

So we use some other DFT code like VASP to calculate the electronic structure, and then return parameters to the ```Optimizer```. The ```Optzimizer``` then uses the forces returned by the ```Calculator``` to move the ionic positions in the structure.

The ```Calculator``` itself can do more than just generate forces for the optimizer. Using the ```.get_vibrations``` can return the vibrational modes calculated by the ```Calculator```. In general, the ```Calculator``` does the hard DFT for ASE, and then ASE maniuplates structures and processes output files.

#### Simple demonstration

We're going to calculate the energy of a bulk FCC metal structure within this notebook. We'll use a computationally inexpensive algorithm built into ASE that will use classical potentials instead of performing DFT on the structure. The calculator is called [EMT](https://wiki.fysik.dtu.dk/ase/ase/calculators/emt.html). First let's get a bulk FCC Cu structure

In [ ]:
from ase.lattice.cubic import FaceCenteredCubic
from ase.visualize import view
from ase.calculators.emt import EMT

atoms = FaceCenteredCubic(directions=[[1,0,0], [0,1,0], [0,0,1]],
                          size=(2,2,2), symbol='Cu', pbc=(1,1,0))
view(atoms)

We then set EMT as our calculator and add the calculator to the ```Atoms``` object. To run the calculator, we'll ask ASE to get the forces on each atom which will also generate the potential energy of the structure.

In [ ]:
calculator = EMT()
atoms.calc = calculator
atoms.get_forces()

Above you'll see that the force vectors on each atom in the lattice were printed

In [ ]:
results = calculator.results
print(results['energy'])

This value that ASE claims is "energy" looks quite different than our DFT energies, which are normally on the order of hundreds of Hartree. If I cut this cell in half by only doubling across two periodic boundaries, I get half the energy.

In [ ]:
atoms = FaceCenteredCubic(directions=[[1,0,0], [0,1,0], [0,0,1]],
                          size=(1,2,2), symbol='Cu', pbc=(1,1,0)) #1/2 the unit cell size
view(atoms)
calculator = EMT()
atoms.calc = calculator
atoms.get_forces()
print(calculator.results['energy'])

So then this energy that ASE has calculated for us is some type of total energy that has been evaluated using a really simple classical analysis technique that we would never use in our research. Instead, we are going to calculate things using JDFTx as our calculator. Let's begin writing a script to do this.

## JDFTx Calculator

We are only going to run single points on our converged Pt_111 structure. Pull the CONTCAR files off of Alpine and store them anywhere in your computer. The CONTCAR is also in the files section of the tutorial.


The next section of code won't work on your local computer. Eventually we'll just copy all of it into a script on Alpine. At the end of this section I'll have a full code block that can be copied.

In [ ]:
from ase.io import read #import read function
import os

path = "C:\\Users\\coopy\\OneDrive - UCB-O365\\Desktop\\temp" #replace this with the path to your CONTCAR file

structure = read(os.path.join(path,"CONTCAR")) #initialize from converged structure
view(structure)

Now we need to initialize our JDFTx calculator. To do so, we're going to reference a scipt called ```JDFTx.py```, which was written by the group that developed JDFTx. This script basically uses python to call JDFTx and formats everything correctly so that we can use it as an ASE calculator. The syntax will be slightly different that the cases above because we are directly calling a script instead of opening a pre-built ASE module.

We need to pass the ```JDFTx.py``` script a few things. We need to give it the path to the psuedopotentials, the ```in``` file tags, the bash execute command, and the calculation directory.

We need to pass the input tags as a list of tuples. We are going to use the input tags from the calculation we did using JDFTx natively. We also need to pass it the executable command that points to the JDFTx installation and gives the number of processes to run.

In [ ]:
jdft_tags = [('elec-n-bands', '192'), #in file tags
             ('kpoint-folding', '6 6 1'),
             ('elec-cutoff', '20 100'),
             ('dump-name', 'Pt_11.$VAR'),
             ('dump End', 'State'),
             ('dump End', 'Ecomponents')
            ]

exe_cmd = 'mpirun -np 8 /home/nisi6161/jdftx_alpine/build/jdftx' #points to JDFTx installation

Now we specify the ID of the pseudopotentials which will point to our pseudopotential path by bash environment variable. We also specify that we want to output files to our current directory.

In [ ]:
pseudos = 'GBRV'
out_file = os.getcwd() #get current path

Now we can use the JDFTx script with all of our input arguments as the calculator, and attach it to the structure.

In [ ]:
from JDFTx import JDFTx #importing the JDFTx.py script for the Calculator

calculator = JDFTx(commands=jdft_tags, executable=exe_cmd, outfile=out_file, pseudoSet = pseudos)
structure.set_calculator(calculator)


The DFT parameters are set up, but we still need to run an optimizer to actually run this calculation. Wer're going to use the [FIRE](https://wiki.fysik.dtu.dk/ase/_modules/ase/optimize/fire.html) optimization algorithm built into ASE. We need to import that package and instantiate it using our output file name and our optimization step size.

In [ ]:
from ase.optimize import FIRE

opt_alpha = 120 # optimization step size

dyn = FIRE(structure, logfile='opt.log', a=(opt_alpha/70) * 0.1) # instantiate optimizer

For some reason we also need to attach trajectory files to the optimizer.

In [ ]:
from ase.io.trajectory import Trajectory

traj = Trajectory('opt.traj', 'w', structure, properties=['energy', 'forces'])
dyn.attach(traj.write, interval=1)

Now we run the calculation with our convergence criteria.

In [ ]:
fmax = 0.04
max_steps = 1 #singlepoint
dyn.run(fmax=fmax, steps=max_steps)
traj.close() # closing trajectory file after writing to it.

Here's the full code block. Copy it into a file called ```calculate.py``` on Alpine.

In [ ]:
#!/usr/bin/env python3
from ase.io import read #import read function
import os
from JDFTx import JDFTx #importing the JDFTx.py script for the Calculator
from ase.optimize import FIRE
from ase.io.trajectory import Trajectory

structure = read("CONTCAR") #initialize from converged structure

jdft_tags = [('elec-n-bands', '192'), #in file tags
             ('kpoint-folding', '6 6 1'),
             ('elec-cutoff', '20 100'),
             ('dump-name', 'Pt_11.$VAR'),
             ('dump', 'End State'),
             ('dump', 'End Ecomponents')
            ]
exe_cmd = 'mpirun -np 8 /home/nisi6161/jdftx_alpine/build/jdftx' #points to JDFTx installation
pseudos = 'GBRV'

calculator = JDFTx(commands=jdft_tags, executable=exe_cmd, outfile=os.getcwd(), pseudoSet = pseudos)
structure.set_calculator(calculator)

opt_alpha = 120 # optimization step size
dyn = FIRE(structure, logfile='opt.log', a=(opt_alpha/70) * 0.1) # instantiate optimizer

traj = Trajectory('opt.traj', 'w', structure, properties=['energy', 'forces'])
dyn.attach(traj.write, interval=1)

fmax = 0.04
max_steps = 1 #singlepoint
dyn.run(fmax=fmax, steps=max_steps)
traj.close() # closing trajectory file after writing to it.

We also need a submission script to tell Alpine to run our script. You can copy the script I have below:
```bash
#!/bin/bash
#SBATCH -J Calculator
#SBATCH --time=23:00:00
#SBATCH -o calculate-%j.out
#SBATCH -e calculate-%j.err
#SBATCH --tasks 8
#SBATCH --nodes 1
#SBATCH --ntasks-per-node 8
#SBATCH --account=ucb-general
#SBATCH --partition amilan
#SBATCH --qos=normal

export JDFTx_NUM_PROCS=2
export I_MPI_FABRICS=shm

module load intel impi

python calculate.py > Pt_111
exit 0
```

Run this submission script in a new directory with the converged Pt_111 CONTCAR and the ```calculate.py``` script. If the script is working, you should see an ```in``` and ```out``` file that the script creates in the calculation directory. If the calculation fails and the slurm error file says it can't find JDFTx.py, add this line to your ```.jdft.bashrc```.
```bash
export PYTHONPATH=$PYTHONPATH:/home/cote3804/Coop_BEAST_DB_Manager/manager/
```

## Why this matters

That certainly seemed cumbersome to run DFT through python! We had to manually specify the same stuff we're used to specifying in the in file, but this time we also had to write a lot more code. The beauty of this method is in the potential for automation. Now that we have a pythonic way of submitting calculations, we can build a large python workflow and submit many calculations in one script.

The group has already written scripts that speed up calculation submission via python. Continue onto the next tutorial called **run_JDFTx** to see how easy it is to submit calculations with python. 